In [1]:
import pyomo.environ as pyo
from pyomo.contrib import appsi
from cpsat import CpsatDirect
# from appsi_cpsat import Cpsat

In [2]:
nodes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

edges = [(0, 1), (0, 2), (0, 3), (1, 4),
         (1, 6), (2, 1), (2, 3), (2, 5),
         (3, 5), (4, 2), (5, 7), (5, 8),
         (6, 4), (6, 7), (6, 9), (7, 4),
         (7, 9), (8, 3), (8, 7), (8, 9)]

distance = {(0, 1): 40, (0, 2):  8, (0, 3): 10, (1, 4):  6,
            (1, 6): 10, (2, 1):  4, (2, 3): 12, (2, 5):  2,
            (3, 5):  1, (4, 2):  2, (5, 7):  4, (5, 8):  3,
            (6, 4):  8, (6, 7): 20, (6, 9):  1, (7, 4):  0,
            (7, 9): 20, (8, 3):  6, (8, 7): 10, (8, 9):  2}

source = 0
sink = 9

In [12]:
model = pyo.ConcreteModel()

In [13]:
model.nodes = pyo.Set(initialize=nodes)
model.edges = pyo.Set(initialize=edges)
model.distance = pyo.Param(model.edges, initialize=distance)
model.x = pyo.Var(edges, domain=pyo.Binary)

In [14]:
@model.Objective(sense=pyo.minimize)
def total_distance(m):
    return sum(m.distance[i, j] * m.x[i, j] for (i, j) in m.edges)

@model.Constraint(model.nodes)
def flow_balance(m, i):
    flow_in =  sum(m.x[j, k] for (j, k) in m.edges if k == i)
    flow_out = sum(m.x[j, k] for (j, k) in m.edges if j == i)

    if i == source:
        return flow_out == 1
    elif i == sink:
        return flow_in == 1
    else:
        return flow_in == flow_out

In [15]:
model.write('test.mps', 'mps', io_options={"symbolic_solver_labels": True})

('test.mps', 4414757744)

In [7]:
solver = pyo.SolverFactory('cpsat')
# solver = pyo.SolverFactory('gurobi')
# solver = pyo.SolverFactory('scip')
# solver = pyo.SolverFactory('appsi_highs')
# solver = pyo.SolverFactory('appsi_gurobi')

# solver._set_instance(model)

In [8]:
result = solver.solve(model, tee=True)


Starting CP-SAT solver v9.6.2534
Parameters: log_search_progress: true
Setting number of workers to 10

Initial optimization model '': (model_fingerprint: 0xd2400b306beaa081)
#Variables: 20 (#bools:19 in objective)
  - 20 Booleans in [0,1]
#kLinear3: 2
#kLinearN: 8 (#terms: 34)

Starting presolve at 0.00s
[ExtractEncodingFromLinear] #potential_supersets=6 #potential_subsets=0 #at_most_one_encodings=0 #exactly_one_encodings=0 #unique_terms=0 #multiple_terms=0 #literals=0 time=5e-06s
[Symmetry] Graph for symmetry has 41 nodes and 51 arcs.
[Symmetry] Symmetry computation done. time: 9e-06 dtime: 5.59e-06
[Probing] deterministic_time: 1.356e-05 (limit: 1) wall_time: 2.7e-05 (20/20)
[Probing] implications and bool_or (work_done=110).
[DetectDuplicateConstraints] #duplicates=0 #without_enforcements=0 time=1.6e-05s
[DetectDominatedLinearConstraints] #relevant_constraints=4 #work_done=54 #num_inclusions=0 #num_redundant=0 time=3e-06s
[ProcessSetPPC] #relevant_constraints=10 #num_inclusions=0 

In [9]:
result

{'Problem': [{'Name': '', 'Lower bound': 15.0, 'Upper bound': 15.0, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 20, 'Number of binary variables': 0, 'Number of integer variables': 20, 'Number of continuous variables': 0, 'Number of nonzeros': None, 'Sense': 1, 'Number of solutions': None}], 'Solver': [{'Status': 'ok', 'Wallclock time': 0.006949, 'Termination condition': 'optimal', 'Termination message': 'An optimal feasible solution was found.'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
for edge, var in model.x.items():
    if var.value != 0:
        print(f'x{edge} = {var.value}')

x(0, 2) = 1
x(2, 5) = 1
x(5, 8) = 1
x(8, 9) = 1


In [11]:
print(pyo.value(model.total_distance))

15
